In [1]:
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score 
from sklearn.pipeline import Pipeline
import numpy as np
import nltk

In [2]:
def write(name, value):
    if type(value) == list:
        value = " ".join([str(x) for x in value])
    with open(name, "w") as fout:
        fout.write(str(value))

In [4]:
movie_reviews

<CategorizedPlaintextCorpusReader in '.../corpora/movie_reviews' (not loaded yet)>

### Данные

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]
texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

LookupError: 
**********************************************************************
  Resource [93mmovie_reviews[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('movie_reviews')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/movie_reviews[0m

  Searched in:
    - 'C:\\Users\\Grigorii/nltk_data'
    - 'C:\\Users\\Grigorii\\anaconda3\\nltk_data'
    - 'C:\\Users\\Grigorii\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Grigorii\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Grigorii\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### Задание 1
Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

In [8]:
def make_pipeline(vectorizer, clf):
    return  Pipeline([("vectorizer", vectorizer), ("classifier", clf)])

In [11]:
pipe_1_a = make_pipeline(CountVectorizer(), LogisticRegression(solver = "liblinear"))
pipe_1_b = make_pipeline(TfidfVectorizer(), LogisticRegression(solver = "liblinear"))


In [14]:
%%time
result_1_a = cross_val_score(pipe_1_a, texts, labels, cv =5)
result_1_b = cross_val_score(pipe_1_b, texts, labels, cv =5)
print("Accuracy первого пайплайна  = %.3f +/- %.3f"%(result_1_a.mean(), result_1_a.std()) )
print("Accuracy второго пайплайна  = %.3f +/- %.3f"%(result_1_b.mean(), result_1_b.std()) )

Accuracy первого пайплайна  = 0.841 +/- 0.017
Accuracy второго пайплайна  = 0.821 +/- 0.004
Wall time: 12.4 s


In [21]:
write("1.txt",  [result_1_a.mean(), result_1_a.std(),result_1_b.mean(), result_1_b.std()])

### Задание 2
Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

In [22]:
pipe_2_a = make_pipeline(CountVectorizer(min_df=10), LogisticRegression(solver = "liblinear"))
pipe_2_b = make_pipeline(CountVectorizer(min_df=50), LogisticRegression(solver = "liblinear"))

In [25]:
%%time
result_2_a = cross_val_score(pipe_2_a, texts, labels, cv = 5).mean()
result_2_b = cross_val_score(pipe_2_b, texts, labels, cv = 5).mean()
print("Accuracy(min_df = 10) = %.4f" % result_2_a )
print("Accuracy(min_df = 50) = %.4f" % result_2_b )

Accuracy(min_df = 10) = 0.8390
Accuracy(min_df = 50) = 0.8130
Wall time: 12.4 s


In [26]:
write("2.txt", [result_2_a,result_2_b])

### Задание 3
Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [27]:
# Игнор предупреждений
import warnings
warnings.filterwarnings('ignore')

In [43]:
pipe_3_a = make_pipeline(CountVectorizer(), LogisticRegression())
pipe_3_b = make_pipeline(CountVectorizer(), LinearSVC())
pipe_3_c = make_pipeline(CountVectorizer(), SGDClassifier())

In [44]:
%%time
result_3_a = cross_val_score(pipe_3_a, texts, labels, cv = 5).mean()
result_3_b = cross_val_score(pipe_3_b, texts, labels, cv = 5).mean()
result_3_c = cross_val_score(pipe_3_c, texts, labels, cv = 5).mean()

print("LogisticRegression accuracy = %.3f" % result_3_a)
print("LinearSVC accuracy = %.3f" % result_3_b)
print("SGDClassifier accuracy = %.3f" % result_3_c)

LogisticRegression accuracy = 0.842
LinearSVC accuracy = 0.833
SGDClassifier accuracy = 0.834
Wall time: 23.2 s


Печально, но это задание надо проходить на Python 2, хотя он уже уходит в историю

In [42]:
min([result_3_a,result_3_b,result_3_c])

0.8325000000000001

In [31]:
write("3.txt", min([result_3_a,result_3_b,result_3_c]))

### Задание 4
Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.


 nltk.download('stopwords')

In [35]:
stop_words_nltk = nltk.corpus.stopwords.words('english')
stop_words_nltk

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [37]:
pipe_4_a = make_pipeline(CountVectorizer(stop_words = stop_words_nltk), LogisticRegression())
pipe_4_b = make_pipeline(CountVectorizer(stop_words= "english"), LogisticRegression())

In [38]:
%%time
result_4_a = cross_val_score(pipe_4_a, texts, labels, cv = 5).mean()
result_4_b = cross_val_score(pipe_4_b, texts, labels, cv = 5).mean()
print("Accuracy(stop_words = nltk) = %.4f" % result_4_a )
print("Accuracy(stop_words = sklearn) = %.4f" % result_4_b )

Accuracy(stop_words = nltk) = 0.8400
Accuracy(stop_words = sklearn) = 0.8365
Wall time: 14.6 s


In [39]:
write("4.txt", [result_4_a, result_4_b] )

### Задание 5
Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.


In [48]:
pipe_5_a = make_pipeline(CountVectorizer(ngram_range = (1,2)), LogisticRegression())
pipe_5_b = make_pipeline(CountVectorizer(analyzer='char_wb', ngram_range=(3, 5)), LogisticRegression())

In [49]:
%%time
result_5_a = cross_val_score(pipe_5_a, texts, labels, cv = 5).mean()
result_5_b = cross_val_score(pipe_5_b, texts, labels, cv = 5).mean()
print("Accuracy(ngram = 2 words) = %.4f" % result_5_a )
print("Accuracy(ngram = 3-5 chars) = %.4f" % result_5_b )

Accuracy(ngram = 2 words) = 0.8545
Accuracy(ngram = 3-5 chars) = 0.8265
Wall time: 2min 15s


In [51]:
write("5.txt", [result_5_a,result_5_b])